<h2 align="center">Importing Libraries</h2>


In [ ]:
import sqlalchemy
from urllib.parse import quote_plus
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
import plotly.express as px

<h2 align="center">Import Data</h2>

In [ ]:
Username = 'root'
Password = 'pasword' # drop your password
Host = 'localhost'
Database = 'codex'
encoded_password = quote_plus(Password)

engine = sqlalchemy.create_engine(f'mysql+mysqldb://{Username}:{encoded_password}@{Host}/{Database}')

<h2 align="center">Initial Exploration</h2>

In [ ]:
Tables = pd.read_sql_query("show tables",engine)["Tables_in_codex"].to_list()
Tables

['brand_marketing',
 'dim_cities',
 'dim_repondents',
 'fact_survey_responses',
 'marketing_types']

In [ ]:
pd.set_option("display.max_columns", None)
for Table in Tables:
    print(Table, "Table")
    display(pd.read_sql_query(F"SELECT * FROM {Table} limit 5", engine))

brand_marketing Table


,Marketing Sub Type,Platform,Platform Rating
0,Influencer Marketing,Athlete Celebs/Brand Ambassador,4.5
1,Social Media Marketing,Athlete & Energetic Social Media Influencers,5.0
2,Content Marketing,Athlete & Energetic Social Media Influencers,5.0
3,Event Marketing,Sunburn Festival,5.0
4,Event Marketing,Red Bull BC One India,5.0


dim_cities Table


,City_ID,City,Tier
0,CT111,Delhi,Tier 1
1,CT112,Mumbai,Tier 1
2,CT113,Bangalore,Tier 1
3,CT114,Chennai,Tier 1
4,CT115,Kolkata,Tier 2


dim_repondents Table


,Respondent_ID,Name,Age,Gender,City_ID
0,120031,Aniruddh Issac,15-18,Female,CT117
1,120032,Trisha Rout,19-30,Male,CT118
2,120033,Yuvraj Virk,15-18,Male,CT116
3,120034,Pranay Chand,31-45,Female,CT113
4,120035,Mohanlal Joshi,19-30,Female,CT120


fact_survey_responses Table


,Response_ID,Respondent_ID,Consume_frequency,Consume_time,Consume_reason,Heard_before,Brand_perception,General_perception,Tried_before,Taste_experience,Reasons_preventing_trying,Current_brands,Reasons_for_choosing_brands,Improvements_desired,Ingredients_expected,Health_concerns,Interest_in_natural_or_organic,Marketing_channels,Packaging_preference,Limited_edition_packaging,Price_range,Purchase_location,Typical_consumption_situations
0,103001,120031,2-3 times a week,To stay awake during work/study,Increased energy and focus,Yes,Neutral,Not sure,No,5,Unfamiliar with the brand,Blue Bull,Availability,Reduced sugar content,Guarana,No,Yes,TV commercials,Compact and portable cans,Yes,50-99,Supermarkets,Studying/working late
1,103002,120032,2-3 times a month,Throughout the day,To boost performance,No,Neutral,Not sure,No,5,Not interested in energy drinks,Bepsi,Availability,More natural ingredients,Caffeine,Yes,Not Sure,Print media,Compact and portable cans,No,50-99,Supermarkets,Sports/exercise
2,103003,120033,Rarely,Before exercise,Increased energy and focus,No,Neutral,Not sure,No,2,Not available locally,Bepsi,Availability,More natural ingredients,Caffeine,No,Yes,Online ads,Innovative bottle design,Not Sure,100-150,Supermarkets,Studying/working late
3,103004,120034,2-3 times a week,To stay awake during work/study,To boost performance,No,Positive,Dangerous,Yes,5,Other,Bepsi,Other,Other,Caffeine,No,Yes,Online ads,Compact and portable cans,No,Above 150,Supermarkets,Sports/exercise
4,103005,120035,Daily,To stay awake during work/study,Increased energy and focus,Yes,Neutral,Effective,Yes,5,Other,Sky 9,Other,More natural ingredients,Caffeine,Yes,Yes,Online ads,Compact and portable cans,Yes,100-150,Online retailers,Studying/working late


marketing_types Table


,Marketing Channels,Marketing Types,Marketing Sub types,Rating
0,Online ads,Digital Marketing,Influencer Marketing,5
1,Online ads,Digital Marketing,Social Media Marketing,4
2,Online ads,Digital Marketing,Content Marketing,4
3,Online ads,Digital Marketing,Search Engine Marketing,3
4,Online ads,Digital Marketing,Email Marketing,3


In [ ]:
for Table in Tables:
    display(pd.read_sql_query(f"SELECT * FROM {Table}", engine).dtypes)

Marketing Sub Type     object
Platform               object
Platform Rating       float64
dtype: object

City_ID    object
City       object
Tier       object
dtype: object

Respondent_ID     int64
Name             object
Age              object
Gender           object
City_ID          object
dtype: object

Response_ID                        int64
Respondent_ID                      int64
Consume_frequency                 object
Consume_time                      object
Consume_reason                    object
Heard_before                      object
Brand_perception                  object
General_perception                object
Tried_before                      object
Taste_experience                   int64
Reasons_preventing_trying         object
Current_brands                    object
Reasons_for_choosing_brands       object
Improvements_desired              object
Ingredients_expected              object
Health_concerns                   object
Interest_in_natural_or_organic    object
Marketing_channels                object
Packaging_preference              object
Limited_edition_packaging         object
Price_range                       object
Purchase_location                 object
Typical_consumption_situations    object
dtype: object

Marketing Channels     object
Marketing  Types       object
Marketing Sub types    object
Rating                  int64
dtype: object

<h2 align="center">Analysis</h2>

### 1. Demographic Insights (examples)

- A: Who prefers energy drink more? (male/female/non-binary?)

In [ ]:
pd.read_sql_query('''
                  WITH GENDER_COMPARISION AS (
                  SELECT F.CONSUME_FREQUENCY, R.GENDER, COUNT(F.RESPONDENT_ID) AS CNT,
                  DENSE_RANK() OVER(PARTITION BY F.CONSUME_FREQUENCY ORDER BY COUNT(F.RESPONDENT_ID) DESC) AS RANK_
                  FROM dim_repondents R JOIN fact_survey_responses F ON R.RESPONDENT_ID = F.RESPONDENT_ID
                  GROUP BY F.CONSUME_FREQUENCY, R.GENDER)

                  SELECT CONSUME_FREQUENCY, GENDER, RANK_ FROM GENDER_COMPARISION
                  WHERE RANK_ = 1
             ''', engine)

,CONSUME_FREQUENCY,GENDER,RANK_
0,2-3 times a month,Male,1
1,2-3 times a week,Male,1
2,Daily,Male,1
3,Once a week,Male,1
4,Rarely,Male,1


In [ ]:
# MALE CONSUME ENERGY DRINK MORE THAN FEMALE IN EVERY CONSUME FREQUENCY

- B: Which age group prefers energy drinks more?

In [ ]:
pd.read_sql_query(''' 
                WITH AGE_GROUP_COMPARISION AS (
                SELECT F.CONSUME_FREQUENCY, D.AGE,
                DENSE_RANK() OVER(PARTITION BY F.CONSUME_FREQUENCY ORDER BY COUNT(D.RESPONDENT_ID) DESC ) AS RANK_ 
                FROM fact_survey_responses F JOIN dim_repondents D ON F.RESPONDENT_ID = D.RESPONDENT_ID
                GROUP BY F.CONSUME_FREQUENCY, D.AGE
                )
                SELECT CONSUME_FREQUENCY, AGE FROM AGE_GROUP_COMPARISION
                WHERE RANK_ = 1

                    ''', engine)

,CONSUME_FREQUENCY,AGE
0,2-3 times a month,19-30
1,2-3 times a week,19-30
2,Daily,19-30
3,Once a week,19-30
4,Rarely,19-30


In [ ]:
# 19-30 AGE GROUP OF PEOPLE IS MORE LIKE TO HAVE ENERGY DRINK

- c: Which type of marketing reaches the most Youth (15-30)?

In [ ]:
pd.read_sql_query(''' 
                  SELECT F.MARKETING_CHANNELS , COUNT(D.RESPONDENT_ID) AS CNT	
                  FROM dim_repondents D JOIN fact_survey_responses F ON D.RESPONDENT_ID = F.RESPONDENT_ID
                  WHERE D.AGE IN ( '15-18', '19-30' )
                  GROUP BY F.MARKETING_CHANNELS
                  ORDER BY CNT DESC
                  LIMIT 1
                  ''', engine)

,MARKETING_CHANNELS,CNT
0,Online ads,3373


In [ ]:
# 'ONLINE ADS' IS MOST POPULAR MARKETING CHANNEL AMONG THE YOUTH (15-30)

### 2. Consumer Preferences:

- A: What are the preferred ingredients of energy drinks among respondents?


In [ ]:
pd.read_sql_query(''' 
                SELECT INGREDIENTS_EXPECTED , COUNT(*) AS CNT
                FROM fact_survey_responses
                GROUP BY INGREDIENTS_EXPECTED
                ORDER BY CNT DESC
                LIMIT 1
                  ''', engine)

,INGREDIENTS_EXPECTED,CNT
0,Caffeine,3896


In [ ]:
# 'CAFFEINE' IS THE MOST PREFERRED INGREDIENTS OF ENEGER DRINKS.

- B: What packaging preferences do respondents have for energy drinks?


In [ ]:
pd.read_sql_query(''' 
                SELECT PACKAGING_PREFERENCE , COUNT(*) AS CNT
                FROM fact_survey_responses
                GROUP BY PACKAGING_PREFERENCE
                ORDER BY CNT DESC
                LIMIT 1
                  ''', engine)

,PACKAGING_PREFERENCE,CNT
0,Compact and portable cans,3984


In [ ]:
# 'COMPACT AND PORTABLE CANS' IS THE MOST PREFERED PACKAGING FOR ENERGY DRINKS.

### 3. Competition Analysis:

- A: Who are the current market leaders?

In [ ]:
pd.read_sql_query(''' 
                SELECT CURRENT_BRANDS , COUNT(*) AS CNT
                FROM fact_survey_responses
                GROUP BY CURRENT_BRANDS
                ORDER BY CNT DESC
                LIMIT 1
                  ''', engine)

,CURRENT_BRANDS,CNT
0,Cola-Coka,2538


In [ ]:
# 'COCA COLA' IS THE MARKET LEADER

- B: What are the primary reasons consumers prefer those brands over ours?


In [ ]:
pd.read_sql_query(''' 
                    SELECT REASONS_FOR_CHOOSING_BRANDS, COUNT(RESPONSE_ID) AS CNT
                    FROM FACT_SURVEY_RESPONSES 
                    GROUP BY REASONS_FOR_CHOOSING_BRANDS
                    ORDER BY CNT DESC
                    LIMIT 1 
                    ''', engine)

,REASONS_FOR_CHOOSING_BRANDS,CNT
0,Brand reputation,2652


In [ ]:
# 'BRAND REPUTATION' IS THE REASON FOR CONSUMER TO PREFRE OTHER BRANDS OVER OURS.

### 4. Marketing Channels and Brand Awareness:


- A: Which marketing channel can be used to reach more customers?

In [ ]:
pd.read_sql_query('''
                  SELECT MARKETING_CHANNELS, COUNT(*) AS CNT 
                  FROM FACT_SURVEY_RESPONSES
                  GROUP BY MARKETING_CHANNELS
                  ORDER BY CNT DESC
                  LIMIT 1
                  ''', engine)

,MARKETING_CHANNELS,CNT
0,Online ads,4020


In [ ]:
# 'ONLINE ADS' IS THE BIGGEST MARKETING CHANNELS

- B: How effective are different marketing strategies and channels in reaching our
customers? By Age Groups


In [ ]:
pd.read_sql_query(''' 
                  WITH MARKETING_STRATEGIES AS (
                  SELECT D.AGE, F.MARKETING_CHANNELS, COUNT(F.RESPONDENT_ID) AS CNT,
                  DENSE_RANK() OVER(PARTITION BY D.AGE ORDER BY COUNT(F.RESPONDENT_ID) DESC) AS RANK_
                  FROM dim_repondents D JOIN fact_survey_responses F ON D.RESPONDENT_ID = D.RESPONDENT_ID
                  WHERE HEARD_BEFORE = 'Yes'
                  GROUP BY D.AGE, F.MARKETING_CHANNELS )
                  SELECT AGE, MARKETING_CHANNELS, RANK_ FROM MARKETING_STRATEGIES
                  WHERE RANK_ =1
                  ''' , engine)

,AGE,MARKETING_CHANNELS,RANK_
0,15-18,Online ads,1
1,19-30,Online ads,1
2,31-45,Online ads,1
3,46-65,Online ads,1
4,65+,Online ads,1


In [ ]:
# 'ONLINE ADS' IS MOST EFFECTIVE MARKETING STRATEGIES.

### 5. Brand Penetration:

- A: What do people think about our brand? (overall rating)

In [ ]:
pd.read_sql_query(''' 
                  SELECT DISTINCT BRAND_PERCEPTION FROM FACT_SURVEY_RESPONSES
                  ''', engine)

,BRAND_PERCEPTION
0,Neutral
1,Positive
2,Negative


In [ ]:
pd.read_sql_query(''' 
                  WITH RATING AS (
                  SELECT BRAND_PERCEPTION, CASE WHEN BRAND_PERCEPTION = "Neutral" THEN 3
                  WHEN BRAND_PERCEPTION = "Positive" THEN 5 ELSE 1 END AS RATING, 
                  COUNT(RESPONDENT_ID) AS CNT FROM FACT_SURVEY_RESPONSES
                  GROUP BY 1
                  )
                  SELECT ROUND((SUM(CNT * RATING)) / SUM(CNT),2) AS OVERALL_RATING
                  FROM RATING
                  ''', engine)

,OVERALL_RATING
0,3.1


In [ ]:
# 'OVERALL RATING' OF OUR BRAND IS 3.1

- B: Which cities do we need to focus more on?


In [ ]:
pd.read_sql_query(''' 
                  SELECT C.CITY, C.TIER, COUNT(S.RESPONDENT_ID) AS CNT
                  FROM dim_cities C JOIN dim_repondents R ON C.CITY_ID = R.CITY_ID
                  JOIN fact_survey_responses S ON S.RESPONDENT_ID = R.RESPONDENT_ID
                  WHERE S.HEARD_BEFORE = 'Yes' AND R.AGE IN ('15-18','19-30')
                  GROUP BY C.CITY, C.TIER
                  HAVING CNT > 100
                  ORDER BY CNT DESC
                   ''', engine)

,CITY,TIER,CNT
0,Bangalore,Tier 1,813
1,Mumbai,Tier 1,651
2,Hyderabad,Tier 1,500
3,Chennai,Tier 1,269
4,Pune,Tier 2,262
5,Delhi,Tier 1,183
6,Ahmedabad,Tier 2,149
7,Kolkata,Tier 2,142
8,Jaipur,Tier 2,101


In [ ]:
px.bar(
    pd.read_sql_query(''' 
                  SELECT C.CITY, C.TIER, COUNT(S.RESPONDENT_ID) AS CNT
                  FROM dim_cities C JOIN dim_repondents R ON C.CITY_ID = R.CITY_ID
                  JOIN fact_survey_responses S ON S.RESPONDENT_ID = R.RESPONDENT_ID
                  WHERE S.HEARD_BEFORE = 'Yes' AND R.AGE IN ('15-18','19-30')
                  GROUP BY C.CITY, C.TIER
                  HAVING CNT > 100
                  ORDER BY CNT DESC
                   ''', engine),
                   x = "CNT", y = "CITY", color_discrete_sequence=["green"],
                   title= "Count of Respondent by Cities"
)

In [ ]:
# COMPANY SHOULD FOCUS ON 'TIER 1' CITY
# WHERE YOUNG BUYERS ARE THERE

### 6. Purchase Behavior:

- A: Where do respondents prefer to purchase energy drinks?


In [ ]:
pd.read_sql_query('''
                  SELECT PURCHASE_LOCATION, COUNT(RESPONDENT_ID) AS CNT
                  FROM fact_survey_responses
                  GROUP BY PURCHASE_LOCATION
                  ORDER BY CNT DESC
                  LIMIT 3
                   ''' , engine)

,PURCHASE_LOCATION,CNT
0,Supermarkets,4494
1,Online retailers,2550
2,Gyms and fitness centers,1464


In [ ]:
px.pie(
    pd.read_sql_query('''
                  SELECT PURCHASE_LOCATION, COUNT(RESPONDENT_ID) AS CNT
                  FROM fact_survey_responses
                  GROUP BY PURCHASE_LOCATION
                  ORDER BY CNT DESC
                  LIMIT 5
                   ''' , engine),
    values = "CNT", labels = "PURCHASE_LOCATION", names = "PURCHASE_LOCATION",
    color_discrete_sequence = px.colors.sequential.RdBu
)

In [ ]:
# RESPONDENT PREERS TO BUY THE DRINK FROM THE SUPERMARKETS & ONLINE RETAILS MORE

- B: What are the typical consumption situations for energy drinks among
respondents?


In [ ]:
pd.read_sql_query(''' 
                  SELECT TYPICAL_CONSUMPTION_SITUATIONS, COUNT(RESPONDENT_ID) AS CNT FROM 
                  FACT_SURVEY_RESPONSES
                  GROUP BY TYPICAL_CONSUMPTION_SITUATIONS
                  ORDER BY CNT DESC
                  ''', engine)

,TYPICAL_CONSUMPTION_SITUATIONS,CNT
0,Sports/exercise,4494
1,Studying/working late,3231
2,Social outings/parties,1487
3,Other,491
4,Driving/commuting,297


In [ ]:
px.bar( pd.read_sql_query(''' 
                  SELECT TYPICAL_CONSUMPTION_SITUATIONS, COUNT(RESPONDENT_ID) AS CNT FROM 
                  FACT_SURVEY_RESPONSES
                  GROUP BY TYPICAL_CONSUMPTION_SITUATIONS
                  ORDER BY CNT DESC
                  ''', engine), y = "TYPICAL_CONSUMPTION_SITUATIONS" , x = "CNT", color_discrete_sequence=["green"],
                  title= "Typical Consumption Situation")

In [ ]:
# 'SPORTS/EXERCISE AND STUDYING/WORKING LATE' IS THE MOST TYPICAL SITUATION FOR DRINKING ENERGY DRINK

- C: What factors influence respondents' such as price range,
limited edition packaging and health concern?


In [ ]:
# PRICE RANGE
pd.read_sql_query(''' 
                  SELECT PRICE_RANGE, COUNT(RESPONDENT_ID) AS RESPONDENT_COUNT
                  FROM FACT_SURVEY_RESPONSES
                  GROUP BY PRICE_RANGE
                  ORDER BY RESPONDENT_COUNT DESC ''', engine)

,PRICE_RANGE,RESPONDENT_COUNT
0,50-99,4288
1,100-150,3142
2,Above 150,1561
3,Below 50,1009


In [ ]:
# '50-99' PRICE RANGE IS MOST POPULAR AMONG RESPONDENT

In [ ]:
# LIMITED EDITION
pd.read_sql_query(''' 
                  SELECT LIMITED_EDITION_PACKAGING, COUNT(RESPONDENT_ID) AS RESPONDENT_COUNT
                  FROM FACT_SURVEY_RESPONSES
                  GROUP BY LIMITED_EDITION_PACKAGING
                  ORDER BY RESPONDENT_COUNT DESC
                   ''', engine)

,LIMITED_EDITION_PACKAGING,RESPONDENT_COUNT
0,No,4023
1,Yes,3946
2,Not Sure,2031


In [ ]:
# 'LIMITED_EDITION_PACKAGING' DOESN'T MATTE FOR MOST OF THE RESPONDENT

In [ ]:
# 'HEALTH CONCERN'
pd.read_sql_query(''' 
                  SELECT HEALTH_CONCERNS, COUNT(RESPONDENT_ID) AS CNT
                  FROM fact_survey_responses
                  GROUP BY HEALTH_CONCERNS
                  ORDER BY CNT DESC
                  ''' , engine)

,HEALTH_CONCERNS,CNT
0,Yes,6045
1,No,3955


In [ ]:
pd.read_sql_query(''' 
                  SELECT 
                    (SELECT COUNT(RESPONDENT_ID) AS CNT FROM fact_survey_responses WHERE HEALTH_CONCERNS = 'Yes') 
                  / COUNT(RESPONDENT_ID) AS PERCENT_OF_PEOPLE
                  FROM fact_survey_responses
                  ''' , engine)

,PERCENT_OF_PEOPLE
0,0.6045


In [ ]:
# ABOUT 60% RESPONDENT CONCERND ABOUT THEIR HEALTH

### 7. Product Development


- A: Which area of business should we focus more on our product development?
(Branding/taste/availability)

In [ ]:
pd.read_sql_query( '''WITH CTE1 AS (
SELECT 
		CASE WHEN TASTE_EXPERIENCE < 4 THEN "BAD_TASTE" ELSE "GOOD_TASTE" END AS EXPERIENCE,
        COUNT(RESPONDENT_ID) AS CNT FROM FACT_SURVEY_RESPONSES
WHERE HEARD_BEFORE = 'Yes' AND TRIED_BEFORE = 'Yes'
GROUP BY CASE WHEN TASTE_EXPERIENCE < 4 THEN "BAD_TASTE" ELSE "GOOD_TASTE" END
ORDER BY CNT DESC
LIMIT 1)
,
CTE2 AS (
SELECT REASONS_PREVENTING_TRYING, COUNT(RESPONDENT_ID) AS CNT 
FROM FACT_SURVEY_RESPONSES
WHERE REASONS_PREVENTING_TRYING NOT IN ('Other', 'Not interested in energy drinks')  AND TRIED_BEFORE = 'No'
GROUP BY REASONS_PREVENTING_TRYING)

SELECT EXPERIENCE AS FEEDBACK , CNT FROM CTE1
UNION ALL
SELECT REASONS_PREVENTING_TRYING AS FEEDBACK, CNT FROM CTE2
ORDER BY CNT DESC
''', engine)

,FEEDBACK,CNT
0,Health concerns,1197
1,Not available locally,1165
2,BAD_TASTE,1129
3,Unfamiliar with the brand,1030


In [ ]:
px.bar(
    pd.read_sql_query(''' 
                  WITH CTE1 AS (
SELECT 
		CASE WHEN TASTE_EXPERIENCE < 4 THEN "BAD_TASTE" ELSE "GOOD_TASTE" END AS EXPERIENCE,
        COUNT(RESPONDENT_ID) AS CNT FROM FACT_SURVEY_RESPONSES
WHERE HEARD_BEFORE = 'Yes' AND TRIED_BEFORE = 'Yes'
GROUP BY CASE WHEN TASTE_EXPERIENCE < 4 THEN "BAD_TASTE" ELSE "GOOD_TASTE" END
ORDER BY CNT DESC
LIMIT 1)
,
CTE2 AS (
SELECT REASONS_PREVENTING_TRYING, COUNT(RESPONDENT_ID) AS CNT 
FROM FACT_SURVEY_RESPONSES
WHERE REASONS_PREVENTING_TRYING NOT IN ('Other', 'Not interested in energy drinks')  AND TRIED_BEFORE = 'No'
GROUP BY REASONS_PREVENTING_TRYING)

SELECT EXPERIENCE AS FEEDBACK , CNT FROM CTE1
UNION ALL
SELECT REASONS_PREVENTING_TRYING AS FEEDBACK, CNT FROM CTE2
ORDER BY CNT DESC
                   ''', engine),
                   x = "CNT", y = "FEEDBACK", color_discrete_sequence=["darkblue"],
                   title= "Count of Respondent by Cities"
)

: 